## Microarray Analysis- Machine Learning Analysis
* Date: April 27, 2022 
* Author: Zeal Jinwala (zsj24)
* Description: In this study, you will analyze a Breast Cancer dataset, GSE7390, and identify a gene signature for prediction of Breast Cancer relapse. Use SVM to predict relapse. Use a forward-selection strategy and 10-fold crossvalidation to determine the best gene signature.

Import packages

In [1]:
import GEOparse
import re
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)

Download and parse data

In [3]:
# Download and parse the dataset. You may use bmes_downloadandparsegse_cached('GSE7390') (which downloads the series file and parses it using geoseriesread()). 
    # You do not need to translate the Probe names to gene IDs; hence, you do not need to download the GPL platform file for this dataset.
gse = GEOparse.get_GEO(geo="GSE7390", destdir=".")

28-Jun-2022 18:58:42 DEBUG utils - Directory . already exists. Skipping.
28-Jun-2022 18:58:42 INFO GEOparse - Downloading ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE7nnn/GSE7390/soft/GSE7390_family.soft.gz to ./GSE7390_family.soft.gz
100%|██████████| 42.7M/42.7M [00:01<00:00, 31.0MB/s]  
28-Jun-2022 18:58:44 DEBUG downloader - Size validation passed
28-Jun-2022 18:58:44 DEBUG downloader - Moving /var/folders/g8/824tqwk50c17j0sh1y69p4z5x108l_/T/tmplhyf9ffu to /Users/zsj24/GitHub/Computational-Analysis/MicroArrayAnalysis/GSE7390_family.soft.gz
28-Jun-2022 18:58:44 DEBUG downloader - Successfully downloaded ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE7nnn/GSE7390/soft/GSE7390_family.soft.gz
28-Jun-2022 18:58:44 INFO GEOparse - Parsing ./GSE7390_family.soft.gz: 
28-Jun-2022 18:58:44 DEBUG GEOparse - DATABASE: GeoMiame
28-Jun-2022 18:58:44 DEBUG GEOparse - SERIES: GSE7390
28-Jun-2022 18:58:44 DEBUG GEOparse - PLATFORM: GPL96
28-Jun-2022 18:58:46 DEBUG GEOparse - SAMPLE: GSM177885
28-Jun-2022 18:

Get target and expression values

In [4]:
# find the patient characteristics, which will be the target in our case
# Cancer relapse status is given as “e.rfs”, with 1 indicating relapse, and 0 indicating no relapse.
cancerRelapseStatus = gse.phenotype_data["characteristics_ch1.14.e.rfs"]

# Expression data
gsedata = None
for gsmid in gse.gsms.keys():
    gsmdata = gse.gsms[gsmid].table.rename(columns={'VALUE':gsmid})
    if gsedata is None: gsedata=gsmdata
    else:
        assert(gsedata['ID_REF'].equals(gsmdata['ID_REF']))
        gsedata = pd.concat([gsedata,gsmdata[gsmid]],axis=1)



Filter 76 significant genes

In [5]:
# Instead of using all of the genes, use only the 76-genes listed in Table 3 of [Gene-expression profiles to predict 
# distant metastasis of lymph-node-negative primary breast cancer.] 
    # Filter out the genes that are not part of the 76-genes, you won't need them for the rest of this assignment.

# sigGenes.txt contains the table from the paper for this study

# reading each line from the text file
sigGenes = []
with open('sigGenes.txt') as f:
    lines = f.readlines()

# Regexp pattern to extract geneIds
pattern = re.compile('\d+_[^\s]*')

# Extracting genes line-by-line
for line in lines:
    gene = re.findall(pattern,str(line))
    if gene:
        sigGenes.append(gene)

# Created a set of extracted genes (76 genes)
sigGenesSet = set([i[0] for i in sigGenes])

# Filter out the genes that are not part of the 76-genes
filteredData = gsedata[gsedata['ID_REF'].isin(sigGenesSet)]


FileNotFoundError: [Errno 2] No such file or directory: 'sigGenes.txt'

Normalize data

In [ ]:
# Normalize data- Standard normalization
step1 = filteredData-filteredData.mean(axis=0)
normalized = step1 /filteredData.std(axis=0)
normalized.drop('ID_REF', inplace=True, axis=1)


Split train and test data

In [ ]:
# Randomly pick out 90% of the samples to serve as training data, and the remaining 10% to serve as test data.
from sklearn.model_selection import train_test_split
x = normalized.values
y = cancerRelapseStatus.values
x_train, x_test, y_train, y_test = train_test_split(np.transpose(x),y,test_size = 0.10)


Generating model


In [ ]:
# Train a SVM model using fitcsvm() on the training data. Note that SVM considers each row as a sample to be predicted and considers each column as features (genes).
from sklearn import svm
clf = svm.SVC(kernel='rbf') # radial base function 
clf.fit(x_train, y_train) 

SVC()


Get model predictions

In [ ]:
# Get SVM predictions using predict() on the test data. Calculate and report the accuracy rate (for a single partition/fold).
y_pred = clf.predict(x_test)

# compTable = np.concatenate([y_pred, y_test, comp])
# Performance for 1-fold
numcorrect = sum(y_pred == y_test)
# numerrors = sum(y_pred != y_test)
accuracy = numcorrect / len(y_test)
# errorrate = numerrors / len(y_test)
print("Accuracy for a single partition fold: \n", accuracy*100)

Accuracy for a single partition fold: 
 75.0



Model evaluation

In [ ]:
# Calculate and report the accuracy (for a single partition/fold), this time by calling the hwmaml_breastcancer_trainandtest() function you wrote.
from breastcancer_trainandtest import *
numerrors = breastcancer_trainandtest(x_train, y_train,x_test,y_test)
print("Accuracy for a single partition/fold using hwmaml_breastcancer_trainandtest(): \n", (1 - numerrors / len(y_test))*100)


Accuracy for a single partition/fold using hwmaml_breastcancer_trainandtest(): 
 75.0


Feature selection

In [ ]:
# Perform forward selection of features (genes) that give the best prediction results (as measured by accuracy). 
# sequentialfs() will do the heavy lifting for you.
# * Create a 10-fold cross-validation of all data samples using cvpartition(). You will pass this on to sequentialfs().
# * Report the names of the genes that were selected by sequentialfs to have the best accuracy.

# Using python's Sequential feature selection for feature selection 
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SequentialFeatureSelector as SFS

clf = svm.SVC(kernel='rbf')
# passing 
sfs = SFS(clf, n_jobs=-1,direction = 'forward',scoring='accuracy',cv = 10)
sfs.fit(np.transpose(x), y)

selectedArray = sfs.get_support()
sigGenesList = np.array(sigGenes)
selectedGenes = sigGenesList[selectedArray]
print(
    "Features selected by forward sequential selection: \n"
    f"{selectedGenes}"
)

print("Number of features selected: \n", len(selectedGenes))

Features selected by forward sequential selection: 
[['219340_s_at']
 ['217771_at']
 ['202418_at']
 ['206295_at']
 ['200726_at']
 ['200965_s_at']
 ['210314_x_at']
 ['217767_at']
 ['219588_s_at']
 ['204073_s_at']
 ['212567_s_at']
 ['211382_s_at']
 ['201663_s_at']
 ['210028_s_at']
 ['218782_s_at']
 ['219724_s_at']
 ['204014_at']
 ['212014_x_at']
 ['218883_s_at']
 ['204888_s_at']
 ['217815_at']
 ['201288_at']
 ['201068_s_at']
 ['214919_s_at']
 ['217471_at']
 ['221816_s_at']
 ['217102_at']
 ['215633_x_at']
 ['204540_at']
 ['209500_x_at']
 ['207118_s_at']
 ['205848_at']
 ['204631_at']
 ['202687_s_at']
 ['220886_at']
 ['221241_s_at']
 ['209862_s_at']
 ['217019_at']]
Number of features selected: 
 38


In [ ]:
# Using the list of genes selected, report the 10-fold cross-validation accuracy of the SVM model.
filterSelected = gsedata[gsedata['ID_REF'].isin(list(selectedGenes))]
xdata = filterSelected.iloc[:,1:]
scores = cross_val_score(clf, np.transpose(xdata) , y, cv=10)
print("The 10-fold cross-validation accuracies of the SVM model: \n", scores)
print("The average 10-fold cross-validation accuracy of the SVM model: \n", np.mean(scores)*100)

The 10-fold cross-validation accuracies of the SVM model: 
 [0.55       0.7        0.55       0.55       0.6        0.6
 0.55       0.45       0.63157895 0.63157895]
The average 10-fold cross-validation accuracy of the SVM model: 
 58.13157894736843
